<a href="https://colab.research.google.com/github/amanjoy10/assignment_amanjoy/blob/main/Fine_tuning_dataset_for_assignment_1.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!pip install peft transformers accelerate
!pip install pytorch_lightning

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 857.3/857.3 kB 22.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 983.2/983.2 kB 59.7 MB/s eta 0:00:00


In [2]:
!pip install datasets

In [2]:
import torch
from datasets import load_dataset
from transformers import TorchAoConfig, AutoProcessor, AutoModelForImageTextToText
import os
import json
import pandas as pd
from transformers import VisionEncoderDecoderModel, DonutProcessor
import torch
from torch.optim import AdamW
from torch.utils.data import Dataset
from PIL import Image
import json
from torch.utils.data import DataLoader

In [3]:
model_name = "naver-clova-ix/donut-base"
processor = DonutProcessor.from_pretrained(model_name)
model = VisionEncoderDecoderModel.from_pretrained(model_name)
device = "cuda" if torch.cuda.is_available() else "cpu"
model.to(device)

/usr/local/lib/python3.12/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


preprocessor_config.json:   0%|          | 0.00/362 [00:00<?, ?B/s]

The image processor of type `DonutImageProcessor` is now loaded as a fast processor by default, even if the model checkpoint was saved with a slow processor. This is a breaking change and may produce slightly different outputs. To continue using the slow processor, instantiate this class with `use_fast=False`. 


config.json: 0.00B [00:00, ?B/s]

tokenizer_config.json:   0%|          | 0.00/518 [00:00<?, ?B/s]

...

pytorch_model.bin:   0%|          | 0.00/809M [00:00<?, ?B/s]

Loading weights:   0%|          | 0/484 [00:00<?, ?it/s]

The tied weights mapping and config for this model specifies to tie decoder.model.decoder.embed_tokens.weight to decoder.lm_head.weight, but both are present in the checkpoints, so we will NOT tie them. You should update the config with `tie_word_embeddings=False` to silence this warning


model.safetensors:   0%|          | 0.00/809M [00:00<?, ?B/s]

VisionEncoderDecoderModel(
  (encoder): DonutSwinModel(
    (embeddings): DonutSwinEmbeddings(
      (patch_embeddings): DonutSwinPatchEmbeddings(
        (projection): Conv2d(3, 128, kernel_size=(4, 4), stride=(4, 4))
      )
      (norm): LayerNorm((128,), eps=1e-05, elementwise_affine=True)
      (dropout): Dropout(p=0.0, inplace=False)
    )
    (encoder): DonutSwinEncoder(
      (layers): ModuleList(
        (0): DonutSwinStage(
          (blocks): ModuleList(
            (0): DonutSwinLayer(
              (layernorm_before): LayerNorm((128,), eps=1e-05, elementwise_affine=True)
              (attention): DonutSwinAttention(
                (self): DonutSwinSelfAttention(
                  (query): Linear(in_features=128, out_features=128, bias=True)
                  (key): Linear(in_features=128, out_features=128, bias=True)
                  (value): Linear(in_features=128, out_features=128, bias=True)
                  (dropout): Dropout(p=0.0, inplace=False)
                )

In [4]:
def build_samples(csv_path, images_root):
    """
    csv_path: path to train.csv
    images_root: path to train/ folder containing subfolders per document
    """

    df = pd.read_csv(csv_path)
    # print(df)

    samples = []

    for _, row in df.iterrows():

        file_name = row["File Name"]
        doc_folder = os.path.join(images_root, file_name)


        label_json = {
            "agreement_value": str(row["Aggrement Value"]),
            "agreement_start_date": str(row["Aggrement Start Date"]),
            "agreement_end_date": str(row["Aggrement End Date"]),
            "renewal_notice_days": str(row["Renewal Notice (Days)"]),
            "party_one": str(row["Party One"]),
            "party_two": str(row["Party Two"])
        }


        if os.path.exists(doc_folder):

            for image_file in os.listdir(doc_folder):

                if image_file.lower().endswith((".png", ".jpg", ".jpeg")):

                    image_path = os.path.join(doc_folder, image_file)

                    sample = {
                        "image_path": image_path,
                        "label": label_json
                    }

                    samples.append(sample)

        else:
            print(f"Warning: Folder not found for {file_name}")

    return samples

In [10]:
CSV_PATH = "/content/drive/MyDrive/assignment_bhavya_data/train_dataset/train.csv"
IMAGES_ROOT = "/content/drive/MyDrive/assignment_bhavya_data/train_dataset"

samples = build_samples(CSV_PATH, IMAGES_ROOT)
print(samples[0])

{'image_path': '/content/drive/MyDrive/assignment_bhavya_data/train_dataset/6683127-House-Rental-Contract-GERALDINE-GALINATO-v2-Page-1/6683127-House-Rental-Contract-GERALDINE-GALINATO-v2-Page-1.jpg', 'label': {'agreement_value': '6500', 'agreement_start_date': '20.05.2007', 'agreement_end_date': '20.05.2008', 'renewal_notice_days': '15.0', 'party_one': 'Antonio Levy S. Ingles, Jr. and/or Mary Rose C. Ingles', 'party_two': 'GERALDINE Q. GALINATO'}}


In [11]:
class DonutDataset(Dataset):
    def __init__(self, samples, processor, max_length=512):
        self.samples = samples
        self.processor = processor
        self.max_length = max_length

    def __len__(self):
        return len(self.samples)

    def __getitem__(self, idx):
        sample = self.samples[idx]

        image = Image.open(sample["image_path"]).convert("RGB")
        label = sample["label"]
        label_str = json.dumps(label, ensure_ascii=False)


        task_prompt = "<s_agreement>"
        target = task_prompt + label_str + "</s_agreement>"
        # print(target)

        pixel_values = self.processor(image, return_tensors="pt").pixel_values.squeeze()

        labels = self.processor.tokenizer(
            target,
            padding="max_length",
            truncation=True,
            max_length=self.max_length,
            return_tensors="pt"
        ).input_ids.squeeze()


        labels[labels == processor.tokenizer.pad_token_id] = -100

        return {
            "pixel_values": pixel_values,
            "labels": labels
        }

In [12]:
model.config.decoder_start_token_id = processor.tokenizer.convert_tokens_to_ids("<s_agreement>")
model.config.pad_token_id = processor.tokenizer.pad_token_id
model.config.eos_token_id = processor.tokenizer.eos_token_id

special_tokens = ["<s_agreement>", "</s_agreement>"]
processor.tokenizer.add_tokens(special_tokens)
model.decoder.resize_token_embeddings(len(processor.tokenizer))

The new embeddings will be initialized from a multivariate normal distribution that has old embeddings' mean and covariance. As described in this article: https://nlp.stanford.edu/~johnhew/vocab-expansion.html. To disable this, use `mean_resizing=False`
The new lm_head weights will be initialized from a multivariate normal distribution that has old embeddings' mean and covariance. As described in this article: https://nlp.stanford.edu/~johnhew/vocab-expansion.html. To disable this, use `mean_resizing=False`


MBartScaledWordEmbedding(57527, 1024, padding_idx=1)

In [13]:
train_dataset = DonutDataset(samples,processor)
train_loader = DataLoader(train_dataset, batch_size=2, shuffle=True)

In [14]:
train_loader

In [15]:
from peft import LoraConfig, get_peft_model

lora_config = LoraConfig(
    r=64,                      # rank
    lora_alpha=16,
    target_modules=[
        "q_proj",
        "v_proj",
        "k_proj",
        "out_proj"
    ],
    lora_dropout=0.1,
    bias="none",
    task_type="SEQ_2_SEQ_LM"
)

peft_model = get_peft_model(model, lora_config)
peft_model.print_trainable_parameters()

trainable params: 4,194,304 || all params: 206,048,376 || trainable%: 2.0356


In [16]:
for param in peft_model.base_model.encoder.parameters():
    param.requires_grad = False

In [17]:
optimizer = AdamW(model.parameters(), lr=1e-4)
peft_model.train()

for epoch in range(50):
    total_loss = 0

    for batch in train_loader:
        pixel_values = batch["pixel_values"].to(device)
        labels = batch["labels"].to(device)

        outputs = model(
            pixel_values=pixel_values,
            labels=labels
        )

        loss = outputs.loss
        loss.backward()

        optimizer.step()
        optimizer.zero_grad()

        total_loss += loss.item()

    print("Epoch:", epoch, "Loss:", total_loss / len(train_loader))

Epoch: 0 Loss: 7.069844563802083
Epoch: 1 Loss: 6.5003470579783125
Epoch: 2 Loss: 5.936944683392842
Epoch: 3 Loss: 5.24077832698822
Epoch: 4 Loss: 4.747280200322469
Epoch: 5 Loss: 4.36392726500829
Epoch: 6 Loss: 3.97268799940745
Epoch: 7 Loss: 3.588298181692759
Epoch: 8 Loss: 3.391861895720164
Epoch: 9 Loss: 3.1218859354654946
Epoch: 10 Loss: 2.8677799304326377
Epoch: 11 Loss: 2.7667829195658364
Epoch: 12 Loss: 2.5462852319081626
Epoch: 13 Loss: 2.3479656477769217
Epoch: 14 Loss: 2.181494971116384
Epoch: 15 Loss: 2.0951093832651773
Epoch: 16 Loss: 2.0057320098082223
Epoch: 17 Loss: 1.8659083048502605
Epoch: 18 Loss: 1.76344233751297
Epoch: 19 Loss: 1.679563154776891
Epoch: 20 Loss: 1.7152923146883647
Epoch: 21 Loss: 1.6584815482298534
Epoch: 22 Loss: 1.5130640864372253
Epoch: 23 Loss: 1.5401743451754253
Epoch: 24 Loss: 1.5097388823827107
Epoch: 25 Loss: 1.4088497310876846
Epoch: 26 Loss: 1.3575067619482677
Epoch: 27 Loss: 1.3515606721242268
Epoch: 28 Loss: 1.2912733952204387
Epoch: 29 

### Test Dataset

In [30]:
TEST_CSV_PATH = "/content/drive/MyDrive/assignment_bhavya_data/test_dataset/test.csv"
TEST_IMAGES_ROOT = "/content/drive/MyDrive/assignment_bhavya_data/test_dataset"

test_samples = build_samples(TEST_CSV_PATH, TEST_IMAGES_ROOT)
test_dataset = DonutDataset(test_samples, processor)
test_loader = DataLoader(test_dataset, batch_size=1, shuffle=False)


In [31]:
import json
from tqdm import tqdm

model.eval()

predictions = []
ground_truths = []

for sample in tqdm(test_samples):

    image = Image.open(sample["image_path"]).convert("RGB")
    pixel_values = processor(image, return_tensors="pt").pixel_values.to(device)

    outputs = model.generate(
        pixel_values=pixel_values,
        max_length=512
    )

    pred_text = processor.batch_decode(outputs, skip_special_tokens=True)[0]

    # Convert prediction string → dict
    try:
        pred_json = json.loads(pred_text)
    except:
        pred_json = {}

    predictions.append(pred_json)
    ground_truths.append(sample["label"])  # this must be dict

100%|██████████| 9/9 [01:04<00:00,  7.19s/it]


In [32]:
print(predictions)
print(ground_truths)

[{}, {}, {}, {}, {}, {}, {}, {}, {}]
[{'agreement_value': '12000', 'agreement_start_date': '01.04.2008', 'agreement_end_date': '31.03.2009', 'renewal_notice_days': '60', 'party_one': 'Hanumaiah ', 'party_two': ' Vishal Bhardwaj '}, {'agreement_value': '9000', 'agreement_start_date': '01.04.2010', 'agreement_end_date': '31.03.2011', 'renewal_notice_days': '30', 'party_one': ' S.Sakunthala', 'party_two': 'V.V.Ravi Kian'}, {'agreement_value': '12000', 'agreement_start_date': '15.12.2012', 'agreement_end_date': '14.11.2013', 'renewal_notice_days': '30', 'party_one': 'V.K.NATARAJ ', 'party_two': ' VYSHNAVI DAIRY SPECIALITIES Private Ltd'}, {'agreement_value': '12000', 'agreement_start_date': '15.12.2012', 'agreement_end_date': '14.11.2013', 'renewal_notice_days': '30', 'party_one': 'V.K.NATARAJ ', 'party_two': ' VYSHNAVI DAIRY SPECIALITIES Private Ltd'}, {'agreement_value': '12000', 'agreement_start_date': '15.12.2012', 'agreement_end_date': '14.11.2013', 'renewal_notice_days': '30', 'party